In [3]:
from src.simulation.simulation import *
import matplotlib.pyplot as plt

import torch
import numpy as np
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast

from src.utils.dataset_utils import *
from src.utils.plot_builder.plot_builder import *
from src.utils.plot_builder.plot_types import *
from src.models.fcnn import FCNN

import os

In [2]:
torch.backends.cudnn.benchmark = True

number_of_frames = 250
discretization_size = 290
d = (0.1, 0.3, 0.3)
N = (number_of_frames, discretization_size, discretization_size)
myu_size = (50, 10, 10)
myu_mstd = (5.4, 0.6)

# Initialize Simulation
sim = Simulation(
    d=d,
    N=N,
    myu_size=myu_size,
    myu_mstd=myu_mstd
)

# Compute state and myu
state, myu = sim.compute()

state = np.abs(state).real
state = state[:, np.newaxis, :, :]
print(f"State Shape: {state.shape}")
print(f"Myu Shape: {myu.shape}")

Computing States: 100%|██████████| 249/249 [00:02<00:00, 88.51it/s]

State Shape: (250, 1, 290, 290)
Myu Shape: (250, 290, 290)


In [ ]:
from src.models.cnn import CNN
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class StateDatasetCNN(torch.utils.data.Dataset):
    def __init__(self, state, myu):
        self.inputs = torch.tensor(state, dtype=torch.float32)  # CNN input: [Batch, Channels, Height, Width]
        self.outputs = torch.tensor(myu, dtype=torch.float32).unsqueeze(1)  # CNN output: Add channel dimension

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        try:
            input_data = self.inputs[idx]
            target_data = self.outputs[idx]
            assert input_data.shape == (1, 290, 290), f"Input shape mismatch: {input_data.shape}"
            assert target_data.shape == (1, 290, 290), f"Target shape mismatch: {target_data.shape}"
            return input_data, target_data
        except Exception as e:
            print(f"Error in __getitem__ for index {idx}: {e}")
            raise


dataset = StateDatasetCNN(state, myu)
dataloader = DataLoader(
    dataset,
    batch_size=1024,
    shuffle=True,
    pin_memory=True,
    num_workers=0
)

hidden_channels = [32, 64, 128, 256]
model = CNN(input_channels=1, hidden_channels=hidden_channels).to(device)

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scaler = GradScaler(enabled=torch.cuda.is_available())

from tqdm import tqdm

# Training Loop
epochs = 5
gradient_accumulation_steps = 4  # Gradient accumulation to simulate larger batches
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    # Use tqdm for the progress bar
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}", leave=True)

    for step, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)

        optimizer.zero_grad()

        # Mixed Precision Training
        with autocast(enabled=True):  # Use updated API without 'args...'
            predictions = model(inputs)
            loss = criterion(predictions, targets) / gradient_accumulation_steps  # Normalize loss

        scaler.scale(loss).backward()

        # Update weights after accumulating gradients
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(dataloader):
            scaler.step(optimizer)
            scaler.update()

        # Update progress bar and total loss
        epoch_loss += loss.item() * gradient_accumulation_steps
        progress_bar.set_postfix({"loss": loss.item() * gradient_accumulation_steps})

    print(f"Epoch {epoch + 1}, Average Loss: {epoch_loss / len(dataloader):.6f}")


torch.save(model.state_dict(), "../src/models/cnn_predictor.pth")
print("Training complete. Model saved to 'cnn_predictor.pth'")

Using device: cuda


C:\Users\meloy\AppData\Local\Temp\ipykernel_20144\1351573408.py:41: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=torch.cuda.is_available())
Epoch 1/5:   0%|          | 0/1 [00:00<?, ?it/s]C:\Users\meloy\AppData\Local\Temp\ipykernel_20144\1351573408.py:61: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):  # Use updated API without 'args...'


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
from src.models.cnn import CNN
from src.simulation.simulation import Simulation
import numpy as np

# Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Simulation Parameters
number_of_frames = 250
discretization_size = 290
d = (0.1, 0.3, 0.3)
N = (number_of_frames, discretization_size, discretization_size)
myu_size = (50, 10, 10)
myu_mstd = (5.4, 0.6)

# Simulation
sim = Simulation(
    d=d,
    N=N,
    myu_size=myu_size,
    myu_mstd=myu_mstd
)
state, myu = sim.compute()

# Preprocess State and Myu
state = np.abs(state).real
state = state[:, np.newaxis, :, :]  # Add channel dimension for CNN
print(f"State Shape: {state.shape}")
print(f"Myu Shape: {myu.shape}")


# Dataset Class for CNN
class StateDatasetCNN(torch.utils.data.Dataset):
    def __init__(self, state, myu):
        self.inputs = torch.tensor(state, dtype=torch.float32)  # CNN input: [Batch, Channels, Height, Width]
        self.outputs = torch.tensor(myu, dtype=torch.float32).unsqueeze(1)  # CNN output: Add channel dimension

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]


# Dataloader
dataset = StateDatasetCNN(state, myu)
dataloader = DataLoader(
    dataset,
    batch_size=1024,  # Reduce batch size if memory is a limitation
    shuffle=True,
    pin_memory=True,
    num_workers=0
)

Using device: cuda


Computing States: 100%|██████████| 249/249 [00:02<00:00, 88.67it/s]

State Shape: (250, 1, 290, 290)
Myu Shape: (250, 290, 290)


In [ ]:
hidden_channels = [32, 64, 128, 256]
model = CNN(input_channels=1, hidden_channels=hidden_channels).to(device)

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scaler = GradScaler(enabled=torch.cuda.is_available())

epochs = 1000
gradient_accumulation_steps = 4
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}", leave=True)

    for step, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)

        optimizer.zero_grad()

        with autocast(enabled=True):
            predictions = model(inputs)
            loss = criterion(predictions, targets) / gradient_accumulation_steps

        scaler.scale(loss).backward()

        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(dataloader):
            scaler.step(optimizer)
            scaler.update()

        epoch_loss += loss.item() * gradient_accumulation_steps
        progress_bar.set_postfix({"loss": loss.item() * gradient_accumulation_steps})

    print(f"Epoch {epoch + 1}, Average Loss: {epoch_loss / len(dataloader):.6f}")

torch.save(model.state_dict(), "../src/models/cnn_predictor.pth")
print("Training complete. Model saved to 'cnn_predictor.pth'")

C:\Users\meloy\AppData\Local\Temp\ipykernel_31280\3111421077.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=torch.cuda.is_available())
Epoch 1/1000:   0%|          | 0/1 [00:00<?, ?it/s]C:\Users\meloy\AppData\Local\Temp\ipykernel_31280\3111421077.py:21: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):
Epoch 1/1000: 100%|██████████| 1/1 [00:09<00:00,  9.32s/it, loss=30.2]


Epoch 1, Average Loss: 30.166006


Epoch 2/1000: 100%|██████████| 1/1 [00:09<00:00,  9.05s/it, loss=30.2]


Epoch 2, Average Loss: 30.166012


Epoch 3/1000: 100%|██████████| 1/1 [00:09<00:00,  9.05s/it, loss=30.2]


...

Epoch 13/1000: 100%|██████████| 1/1 [00:09<00:00,  9.09s/it, loss=1.35]


Epoch 13, Average Loss: 1.349118


Epoch 14/1000: 100%|██████████| 1/1 [00:09<00:00,  9.08s/it, loss=1]


Epoch 14, Average Loss: 0.999657


Epoch 15/1000:   0%|          | 0/1 [00:00<?, ?it/s]